# Classification using Transfer Learning using PyTorch & NVIDIA DALI

## Pneumonia Detection

### Check GPU Runtime

In [1]:
!nvidia-smi

Thu Apr 15 03:47:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Installing NVIDIA-DALI

In [2]:
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-cuda110

Looking in indexes: https://pypi.org/simple, https://developer.download.nvidia.com/compute/redist


### Downloading Data From Kaggle

- Download Kaggle API key from your Kaggle Account. Go to www.kaggle.com -> My Account -> Create New API token
- Place the file in the working directory and execute next cell

In [3]:
! mkdir ~/.kaggle
! mv kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
mv: cannot stat 'kaggle.json': No such file or directory


### Importing Required Libraries

In [4]:
import os
import shutil
import sys
import platform
import time

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

from nvidia.dali import pipeline_def, Pipeline
import nvidia.dali.fn as fn
import nvidia.dali.types as types
from nvidia.dali.plugin.pytorch import DALIGenericIterator, DALIClassificationIterator

import torchvision.utils as utils
import torch
import torchvision
import skimage.transform

### Center cropping and Placing Data in Correct Place

In [5]:
def image_center_crop(path):
    """
    Makes a square center crop of an img, which is a [h, w, 3] numpy array.
    Returns [min(h, w), min(h, w), 3] output with same width and height.
    For cropping use numpy slicing.
    """

    image = Image.open(path)
    img = np.asarray(image)

    shape = img.shape
    dim = len(shape)

    if dim == 3:
        h, w, c = shape
    else:
        h, w = shape

    h_crop = min(h, w)

    if dim == 3:
        cropped_img = img[
            ...,
            (h // 2 - h_crop // 2) : (h // 2 + h_crop // 2),
            (w // 2 - h_crop // 2) : (w // 2 + h_crop // 2),
            :,
        ]
        image = Image.fromarray(cropped_img, "RGB")
    else:
        cropped_img = img[
            ...,
            (h // 2 - h_crop // 2) : (h // 2 + h_crop // 2),
            (w // 2 - h_crop // 2) : (w // 2 + h_crop // 2),
        ]
        image = Image.fromarray(cropped_img)

    image.save(path)

In [6]:
if not os.path.exists("./data"):
    os.mkdir("./data")

if not os.path.exists("./data/coronahack-chest-xraydataset.zip"):
    os.chdir("./data")
    os.system("kaggle datasets download -d praveengovi/coronahack-chest-xraydataset")
    os.chdir("..")
else:
    print("Download found...")

print("Starting Extraction")
print(platform.system())
if platform.system() in ["Linux", "Darwin"]:
    os.system('unzip -q "./data/coronahack-chest-xraydataset.zip" -d "./data/"')
else:
    os.system(
        'tar -xf "./data/coronahack-chest-xraydataset.zip" --directory "./data/" '
    )
print("Extraction Complete")

print("Reading Metadata")
images_data = pd.read_csv("./data/Chest_xray_Corona_Metadata.csv")

os.mkdir("./data/Corona_Classification_data")
os.mkdir("./data/Corona_Classification_data/train")
os.mkdir("./data/Corona_Classification_data/train/INFECTED")
os.mkdir("./data/Corona_Classification_data/train/NORMAL")
os.mkdir("./data/Corona_Classification_data/test")
os.mkdir("./data/Corona_Classification_data/test/NORMAL")
os.mkdir("./data/Corona_Classification_data/test/INFECTED")


print("Starting Preprocessing and Moving According to Labels...")

for index, row in images_data.iterrows():
    if row["Dataset_type"] == "TRAIN":
        path_of_image = f"./data/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/{row['X_ray_image_name']}"
        image_center_crop(path_of_image)
        if row["Label"] == "Normal":
            shutil.move(
                path_of_image,
                f"./data/Corona_Classification_data/train/NORMAL/{row['X_ray_image_name']}",
            )

        if row["Label"] == "Pnemonia":
            shutil.move(
                path_of_image,
                f"./data/Corona_Classification_data/train/INFECTED/{row['X_ray_image_name']}",
            )

    if row["Dataset_type"] == "TEST":
        path_of_image = f"./data/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test/{row['X_ray_image_name']}"
        image_center_crop(path_of_image)
        if row["Label"] == "Normal":
            shutil.move(
                path_of_image,
                f"./data/Corona_Classification_data/test/NORMAL/{row['X_ray_image_name']}",
            )

        if row["Label"] == "Pnemonia":
            shutil.move(
                path_of_image,
                f"./data/Corona_Classification_data/test/INFECTED/{row['X_ray_image_name']}",
            )
    sys.stdout.write(f"\rCropping Successfull for {row['X_ray_image_name']}")
    sys.stdout.flush()


print("Moving Complete")
shutil.rmtree("./data/Coronahack-Chest-XRay-Dataset")

files_to_be_deleted = [
    "1-s2.0-S1684118220300682-main.pdf-002-a1.png",
    "1-s2.0-S1684118220300682-main.pdf-002-a2.png",
    "1-s2.0-S1684118220300682-main.pdf-003-b1.png",
    "1-s2.0-S1684118220300682-main.pdf-003-b2.png",
    "7EF28E12-F628-4BEC-A8C5-E6277C2E4F60.png",
    "23E99E2E-447C-46E5-8EB2-D35D12473C39.png",
    "41591_2020_819_Fig1_HTML.webp-day5.png",
    "41591_2020_819_Fig1_HTML.webp-day10.png",
]

for filename in files_to_be_deleted:
    os.remove(f"./data/Corona_Classification_data/train/INFECTED/{filename}")
print("Cleaning Complete")

Starting Extraction
Linux
Extraction Complete
Reading Metadata
Starting Preprocessing and Moving According to Labels...
Cropping Successfull for person1632_virus_2827.jpegMoving Complete
Cleaning Complete


### Creating NVIDIA DALI PipeLine

In [7]:
@pipeline_def
def HybridPipelineTrain(num_gpus, root_path, output_size):
    device_id = Pipeline.current().device_id
    jpegs, labels = fn.readers.file(
        name='Reader', file_root =root_path, random_shuffle=True, shard_id=device_id, num_shards=num_gpus
        )
    images = fn.decoders.image(jpegs, device='mixed')
    images = fn.flip(images, horizontal = fn.random.coin_flip(), vertical = fn.random.coin_flip(), device = "gpu")
    images = fn.rotate(images, angle=fn.random.uniform(range=(-90., 90.)), device = "gpu")
    images = fn.random_resized_crop(images,
            size = output_size,
            random_area = [0.4, 1.0],
            random_aspect_ratio = [0.5, 1.5],
            device="gpu",
            )
    images = fn.crop_mirror_normalize(
            images,
            dtype=types.FLOAT,
            mean=[0.0, 0.0, 0.0],
            std=[255., 255., 255.],
            )

    return images, labels

In [8]:
@pipeline_def
def HybridPipelineTest(num_gpus, root_path, output_size):
    device_id = Pipeline.current().device_id
    jpegs, labels = fn.readers.file(
        name='Reader', file_root =root_path, random_shuffle=True, shard_id=device_id, num_shards=num_gpus
        )
    images = fn.decoders.image(jpegs, device='mixed')
    images = fn.resize(images, size=output_size)
    images = fn.crop_mirror_normalize(
            images,
            dtype=types.FLOAT,
            mean=[0.0, 0.0, 0.0],
            std=[255., 255., 255.],
            )
    return images, labels

### Checking PipeLine Validity

In [9]:
!mkdir Augmentation_Check

In [10]:
label_range = (0, 1)
pipes = [HybridPipelineTrain(batch_size=64, num_threads=2, device_id=device_id, num_gpus=1, 
                             root_path="/content/data/Corona_Classification_data/train",
                             output_size = (224,224)
                             ) for device_id in range(1)]

for pipe in pipes:
    pipe.build()

dali_iter = DALIGenericIterator(pipes, ['data', 'label'], reader_name='Reader')

count = 0
for i, data in enumerate(dali_iter):
    # Testing correctness of labels
    for d in data:
        label = d["label"]
        image = d["data"]
        grid = utils.make_grid(image)
        plt.imshow(grid.cpu().numpy().transpose((1, 2, 0)))
        plt.savefig(f"/content/Augmentation_Check/{count}.png")
        plt.close()
        count+=1
        ## labels need to be integers
        assert(np.equal(np.mod(label, 1), 0).all())
        ## labels need to be in range pipe_name[2]
        assert((label >= label_range[0]).all())
        assert((label <= label_range[1]).all())

dali_iter.reset()
print("OK")

OK


### Model and Training Code

In [11]:
# Dictionary for pretrained models and their last layer name
pretrained_models = {
    "ResNet18": [torchvision.models.resnet18, "layer4", (224, 224)],
    "ResNet34": [torchvision.models.resnet34, "layer4", (224, 224)],
    "ResNet50": [torchvision.models.resnet50, "layer4", (224, 224)],
    "ResNet101": [torchvision.models.resnet101, "layer4", (224, 224)],
    "ResNet152": [torchvision.models.resnet152, "layer4", (224, 224)],
    "Alexnet": [torchvision.models.alexnet, "features", (256, 256)],
    "VGG11": [torchvision.models.vgg11_bn, "features", (224, 224)],
    "VGG13": [torchvision.models.vgg13_bn, "features", (224, 224)],
    "VGG16": [torchvision.models.vgg16_bn, "features", (224, 224)],
    "VGG19": [torchvision.models.vgg19_bn, "features", (224, 224)],
    "GoogleNet": [torchvision.models.googlenet, "inception5b", (224, 224)],
    "Inception": [torchvision.models.inception_v3, "Mixed_7c", (299, 299)],
}

In [12]:
# The main model object
class PneumoniaDetection:
    """
    Model Architecture and Forward Training Path for the Pneumonia Detection
    Idea is to use transfer Learning
    """

    def __init__(self, base_model="ResNet18", colab=False):
        assert base_model in [
            "ResNet18",
            "ResNet34",
            "ResNet50",
            "ResNet101",
            "ResNet152",
            "Alexnet",
            "VGG11",
            "VGG13",
            "VGG16",
            "VGG19",
            "GoogleNet",
            "Inception",
        ]

        # saving base model name to use it in saving the model
        self.base_model = base_model
        if colab:
            self.colab_training = f"drive/My Drive/{self.base_model}"
        else:
            self.colab_training = "."

        if not os.path.exists(f"{self.colab_training}/model"):
            os.mkdir(f"{self.colab_training}/model")
        if not os.path.exists(f"{self.colab_training}/model_results"):
            os.mkdir(f"{self.colab_training}/model_results")

        if os.path.exists(f"{self.colab_training}/model/ConvModel_{self.base_model}"):
            # check if the model is intialized before
            self.model = torch.load(
                f"{self.colab_training}/model/ConvModel_{self.base_model}"
            )
        else:
            # If not initialized before
            # Download it and save it
            self.model = pretrained_models[self.base_model][0](pretrained=True)
            for name, param in self.model.named_parameters():
                param.requires_grad = True

            # Modify last Fully Connected layer to predict for
            # Our requirements
            if self.base_model in ["Alexnet", "VGG11", "VGG13", "VGG16", "VGG19"]:
                num_ftrs = self.model.classifier[6].in_features
                self.model.classifier[6] = torch.nn.Linear(num_ftrs, 2)
            else:
                self.model.fc = torch.nn.Sequential(
                    torch.nn.Linear(self.model.fc.in_features, 500),
                    torch.nn.ReLU(),
                    torch.nn.Dropout(),
                    torch.nn.Linear(500, 2),
                )

            # Save model
            torch.save(
                self.model, f"{self.colab_training}/model/ConvModel_{self.base_model}"
            )

        # get final model for using it in Class Activation Map
        self.final_layer = self.model._modules.get(
            pretrained_models[self.base_model][1]
        )

        # Different image transformations for training, testing and displaying
        self.train_transformation = torchvision.transforms.Compose(
            [
                torchvision.transforms.RandomHorizontalFlip(),
                torchvision.transforms.RandomVerticalFlip(),
                torchvision.transforms.RandomRotation(25),
                torchvision.transforms.RandomResizedCrop(
                    pretrained_models[self.base_model][2],
                    scale=(0.4, 1.0),
                    ratio=(0.5, 1.5),
                    interpolation=2,
                ),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize(
                    mean=[0,0,0], std=[255,255,255]
                ),
            ]
        )
        self.test_transformation = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize(pretrained_models[self.base_model][2]),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize(
                    mean=[0,0,0], std=[255,255,255]
                ),
            ]
        )
        self.display_transformation = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize(pretrained_models[self.base_model][2]),
            ]
        )

    def train(
        self,
        optimizer,
        loss_fun,
        train_data,
        test_data,
        epochs=20,
        early_stopping_threshold=4,
        device="cuda",
        dali = False
    ):
        """
        Train function:
        parameters:
        optimizer   : optimizer object
        loss_fun    : Loss Function object
        train_data  : train dataloader
        test_data   : test  dataloader
        epochs      : default value 20
        early_stopping_threshold : Early stopping threshold
        device      : 'cuda' or 'cpu', default 'cuda'
        """

        # transfer model to device available
        self.model.to(device)

        max_accurracy = 0.0
        train_losses = []
        test_losses = []
        train_accuracies = []
        test_accuracies = []

        for epoch in range(epochs):
            start = time.time()

            training_loss = 0.0

            train_correct = 0
            train_total = 0

            # Training over batches
            self.model.train(mode=True)
            for i, train_batch in enumerate(train_data):
                if dali:
                    d = train_batch[0]
                    train_images, train_labels = d["data"], d["label"]
                    # train_images = train_images.permute(0,3,1,2)
                    train_images = train_images.to(device)
                    train_labels = train_labels.to(device, dtype=torch.int64).squeeze()
                else:
                    train_images, train_labels = train_batch
                    train_images = train_images.to(device)
                    train_labels = train_labels.to(device)

                optimizer.zero_grad()
                train_output = self.model(train_images)

                if self.base_model == "Inception":
                    train_output = train_output.logits

                train_loss = loss_fun(train_output, train_labels)

                train_loss.backward()
                optimizer.step()

                training_loss += train_loss.item()

                _, train_predicted = torch.max(train_output.data, 1)

                train_total += train_labels.size(0)
                train_ccount = (train_predicted == train_labels).sum().item()
                train_correct += train_ccount
                
                text = !nvidia-smi
                text = text[9][60:65]
                
                sys.stdout.write(
                    f"\rEpoch {epoch+1:03d}\t"
                    f"Train Loss => {train_loss:08.4f} "
                    f"Train Accuracy => "
                    f"{train_ccount/train_images.shape[0]*100:06.2f} "
                    f"GPU Utilization => {text}"
                )
            
            if dali:
                train_data.reset()

            training_accuracy = train_correct / train_total * 100

            valid_loss = 0.0
            test_correct = 0
            test_total = 0
            misses = 0
            previous_accuracy = 0
            tp = 0
            fp = 0
            fn = 0

            # Test over batches
            self.model.train(mode=False)
            with torch.no_grad():
                for i, test_batch in enumerate(test_data):
                    if dali:
                        d = test_batch[0]
                        test_images, test_labels = d["data"], d["label"]
                        # test_images = test_images.permute(0,3,1,2)
                        test_images = test_images.to(device)
                        test_labels = test_labels.to(device, dtype=torch.int64).squeeze()
                    else:
                        test_images, test_labels = test_batch
                        test_images = test_images.to(device)
                        test_labels = test_labels.to(device)

                    test_output = self.model(test_images)

                    test_loss = loss_fun(test_output, test_labels)
                    valid_loss += test_loss.item()

                    _, test_predicted = torch.max(test_output.data, 1)

                    test_total += test_labels.size(0)
                    test_ccount = (test_predicted == test_labels).sum().item()
                    tp += ((test_labels == 1) & ((test_predicted) == 1)).sum().item()
                    fn += ((test_labels != 0) & ((test_predicted) == 0)).sum().item()
                    fp += ((test_labels != 1) & ((test_predicted) == 1)).sum().item()
                    test_correct += test_ccount

            testing_accuracy = test_correct / test_total * 100
            testing_precision = tp / (tp + fp) * 100
            testing_recall = tp / (tp + fn) * 100
            testing_f1 = (
                2.0
                * testing_recall
                * testing_precision
                / (testing_recall + testing_precision)
            )
            if dali:
                test_data.reset()

            sys.stdout.flush()
            sys.stdout.write("\r")

            time_taken = time.time() - start

            print(
                f"Epoch {epoch + 1:03d}\n"
                f"Train Loss => {training_loss:08.4f} "
                f"Train Accuracy => {training_accuracy:06.2f} "
                f"Test Loss => {valid_loss:08.4f} "
                f"Test Accuracy => {testing_accuracy:06.2f} "
                f"Test Precision => {testing_precision:06.2f}\n"
                f"Test Recall => {testing_recall:06.2f} "
                f"Test F1 Score => {testing_f1:06.2f} "
                f"Time Taken => {time_taken:08.4f}"
                f"\n{'='*150}"
            )

            train_losses.append(training_loss)
            test_losses.append(valid_loss)
            train_accuracies.append(training_accuracy)
            test_accuracies.append(testing_accuracy)

            # Save if it is better model than max_accuracy
            if testing_accuracy > max_accurracy:
                max_accurracy = testing_accuracy
                torch.save(
                    self.model,
                    f"{self.colab_training}/model/ConvModel_{self.base_model}",
                )

                with open(
                    f"{self.colab_training}/model_results/ConvModel_{self.base_model}_results.txt",
                    "w",
                ) as f:
                    f.writelines(
                        [
                            f"BaseModel: {self.base_model}\n",
                            f"Epochs: {epoch + 1:03d}\n",
                            f"Train Dataloader Batch Size: {train_data.batch_size}\n",
                            f"Test Dataloader Batch Size: {test_data.batch_size}\n",
                            f"Params for Optimizer: {optimizer.__repr__()}\n",
                            f"Train Loss: {training_loss:08.4f}\n",
                            f"Test Loss: {valid_loss:08.4f}\n",
                            f"Train Accuracy: {training_accuracy:06.2f}\n",
                            f"Test Accuracy: {testing_accuracy:06.2f}\n",
                            f"Test Precision: {testing_precision:06.2f}\n",
                            f"Test Recall: {testing_recall:06.2f}\n",
                            f"Test F1 Score: {testing_f1:06.2f}\n",
                            f"Time Taken: {time_taken:08.4f} seconds",
                        ]
                    )

            # Decide and stop early if needed
            if epoch >= 1:
                if (
                    previous_accuracy > testing_accuracy
                    and misses < early_stopping_threshold
                ):
                    misses += 1
                    previous_accuracy = testing_accuracy
                elif previous_accuracy > testing_accuracy:
                    print(f"Early Stopping....")
                    print(
                        f"Epoch {epoch + 1:03d}\t"
                        f"Train Loss => {training_loss:08.4f} "
                        f"Train accuracy => {training_accuracy:06.2f} "
                        f"Test Loss => {valid_loss:08.4f} "
                        f"Test Accuracy => {testing_accuracy:06.2f} "
                        f"Test Precision => {testing_precision:06.2f} "
                        f"Test Recall => {testing_recall:06.2f} "
                        f"Test F1 Score => {testing_f1:06.2f} "
                        f"Time Taken => {time_taken:08.4f}"
                    )
                    break
            previous_accuracy = testing_accuracy

            np.save(
                f"{self.colab_training}/model/train_losses_{self.base_model}",
                train_losses,
            )
            np.save(
                f"{self.colab_training}/model/train_accuracies_{self.base_model}",
                train_accuracies,
            )
            np.save(
                f"{self.colab_training}/model/test_losses_{self.base_model}",
                test_losses,
            )
            np.save(
                f"{self.colab_training}/model/test_accuracies_{self.base_model}",
                test_accuracies,
            )

    def test(self, loss_fun, test_data, device="cuda", has_labels=False, dali=False):
        print("Starting Evaluating....")
        start = time.time()
        self.model.eval()
        test_loss = 0.0
        correct = 0
        total = 0
        tp = 0
        fp = 0
        fn = 0
        predictions = []

        # Without changing parameters
        with torch.no_grad():
            # Testing over batches
            for i, batch in enumerate(test_data):
                if dali:
                    d = batch[0]
                    test_images, test_labels = d["data"], d["label"]
                    # test_images = test_images.permute(0,3,1,2)
                    test_images = test_images.to(device)
                    if has_labels:
                        test_labels = test_labels.to(device, dtype=torch.int64).squeeze()
                else:
                    test_images, test_labels = batch
                    test_images = test_images.to(device)
                    if has_labels:
                        test_labels = test_labels.to(device)

                output = self.model(test_images)
                if has_labels:
                    loss = loss_fun(output, test_labels)
                    test_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                predictions.extend(list(np.asarray(predicted.to("cpu"))))
                if has_labels:
                    total += test_labels.size(0)
                    correct += (predicted == test_labels).sum().item()
                    tp += ((test_labels == 1) & ((predicted) == 1)).sum().item()
                    fn += ((test_labels != 0) & ((predicted) == 0)).sum().item()
                    fp += ((test_labels != 1) & ((predicted) == 1)).sum().item()

        if has_labels:
            testing_accuracy = correct / total * 100
            testing_precision = tp / (tp + fp) * 100
            testing_recall = tp / (tp + fn) * 100
            testing_f1 = (
                2.0
                * testing_recall
                * testing_precision
                / (testing_recall + testing_precision)
            )

            print(
                f"Test Loss => {test_loss:08.4f} "
                f"Test accuracy => {testing_accuracy:06.2f} "
                f"Test Precision => {testing_precision:06.2f} "
                f"Test Recall => {testing_recall:06.2f} "
                f"Test F1 Score => {testing_f1:06.2f} "
                f"Time Taken => {time.time() - start:08.4f}"
            )

        return predictions

    def CAM(self, image_path_input, overlay_path_output, device="cuda"):
        """
        CAM - Class Activation Map
        """

        # open image
        image = Image.open(image_path_input)
        image = image.convert("RGB")
        print(image.mode)

        tensor = self.test_transformation(image)

        prediction_var = torch.autograd.Variable(
            (tensor.unsqueeze(0)).cuda(), requires_grad=True
        )
        self.model.to(device)
        self.model.eval()

        class SaveFeatures:
            features = None

            def __init__(self, m):
                self.hook = m.register_forward_hook(self.hook_fn)

            def hook_fn(self, module, input, output):
                self.features = ((output.cpu()).data).numpy()

            def remove(self):
                self.hook.remove()

        activated_features = SaveFeatures(self.final_layer)
        prediction_var = prediction_var.to(device)
        prediction = self.model(prediction_var)

        pred_probabilities = torch.nn.functional.softmax(
            prediction, dim=0
        ).data.squeeze()

        activated_features.remove()

        torch.topk(pred_probabilities, 1)

        def getCAM(feature_conv, weight_fc, class_idx):
            _, nc, h, w = feature_conv.shape
            cam = weight_fc[class_idx].dot(feature_conv.reshape((nc, h * w)))
            cam = cam.reshape(h, w)
            cam = cam - np.min(cam)
            cam_img = cam / np.max(cam)
            return [cam_img]

        weight_softmax_params = list(self.model._modules.get("fc").parameters())
        weight_softmax = np.squeeze(weight_softmax_params[0].cpu().data.numpy())

        class_idx = torch.topk(pred_probabilities, 1)[1].int()

        overlay = getCAM(activated_features.features, weight_softmax, class_idx)

        plt.figure(figsize=(32, 15))
        plt.subplot(1, 2, 1)
        plt.imshow(self.display_transformation(image))
        plt.subplot(1, 2, 2)
        plt.imshow(self.display_transformation(image))
        plt.imshow(
            skimage.transform.resize(overlay[0], tensor.shape[1:3]),
            alpha=0.4,
            cmap="jet",
        )
        plt.savefig(overlay_path_output)

In [13]:
if torch.cuda.is_available():
    print("Using GPU")
    device = torch.device("cuda")
else:
    print("Using CPU")
    device = torch.device("cpu")

Using GPU


### Hyperparameters

In [14]:
batch_size = 64
learning_rate = 0.0001
dali = True
N_GPUS = 1
epochs = 10
base_model = "ResNet50"
optimizer_name = 'Adam'
colab = False
train_data_path = "./data/Corona_Classification_data/train/"
test_data_path = "./data/Corona_Classification_data/test/"

In [15]:
print("Creating Model Object: ")
model = PneumoniaDetection(base_model, colab=colab)

Creating Model Object: 


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:804: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


### DataLoader

In [16]:
if dali:
    pipes = [HybridPipelineTrain(batch_size=batch_size, num_threads=2, device_id=device_id, num_gpus=N_GPUS, 
                             root_path=train_data_path,
                             output_size = pretrained_models[base_model][2]
                             ) for device_id in range(N_GPUS)]

    for pipe in pipes:
        pipe.build()

    train_data_loader = DALIGenericIterator(pipes, ['data', 'label'], reader_name='Reader')

    pipes2 = [HybridPipelineTest(batch_size=batch_size, num_threads=2, device_id=device_id, num_gpus=N_GPUS, 
                             root_path=test_data_path,
                             output_size = pretrained_models[base_model][2]
                             ) for device_id in range(N_GPUS)]
    for pipe in pipes2:
        pipe.build()

    test_data_loader = DALIGenericIterator(pipes, ['data', 'label'], reader_name='Reader')
else:
    print("Setting up Data Directories")
    
    train_data = torchvision.datasets.ImageFolder(
        root=train_data_path, transform=model.train_transformation
    )
    
    test_data = torchvision.datasets.ImageFolder(
        root=test_data_path, transform=model.test_transformation
    )    

    train_data_loader = torch.utils.data.DataLoader(
        train_data, batch_size=batch_size, shuffle=True
    )
    test_data_loader = torch.utils.data.DataLoader(
        test_data, batch_size=batch_size, shuffle=True
    )

### Optimizer and Training

In [17]:
optimizers = {
    "Adam": torch.optim.Adam,
    "SGD": torch.optim.SGD,
    "RMSprop": torch.optim.RMSprop,
    "Adagrad": torch.optim.Adagrad,
    "Adadelta": torch.optim.Adadelta,
}

In [18]:
optimizer = optimizers[optimizer_name](
    model.model.parameters(), lr=learning_rate
)

print("Starting Training")    
model.train(
    optimizer,
    torch.nn.CrossEntropyLoss(),
    train_data_loader,
    test_data_loader,
    epochs=epochs,
    device=device,
    dali=dali
)
print("Completed Training")

Starting Training
Epoch 001
Train Loss => 015.6931 Train Accuracy => 091.51 Test Loss => 031.6230 Test Accuracy => 085.84 Test Precision => 098.39
Test Recall => 045.12 Test F1 Score => 061.87 Time Taken => 062.9397
Epoch 002
Train Loss => 009.5347 Train Accuracy => 095.62 Test Loss => 009.9258 Test Accuracy => 095.22 Test Precision => 093.84
Test Recall => 086.87 Test F1 Score => 090.22 Time Taken => 062.2051
Epoch 003
Train Loss => 007.3890 Train Accuracy => 096.59 Test Loss => 007.4554 Test Accuracy => 096.59 Test Precision => 089.86
Test Recall => 097.63 Test F1 Score => 093.58 Time Taken => 062.7988
Epoch 004
Train Loss => 006.2090 Train Accuracy => 097.24 Test Loss => 017.9958 Test Accuracy => 092.19 Test Precision => 076.95
Test Recall => 098.72 Test F1 Score => 086.49 Time Taken => 062.0153
Epoch 005
Train Loss => 006.7074 Train Accuracy => 097.10 Test Loss => 005.3144 Test Accuracy => 097.44 Test Precision => 096.57
Test Recall => 093.30 Test F1 Score => 094.91 Time Taken => 0